In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

## Import Header Files

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pickle
import os
import re
import numpy as np
import pandas as pd

In [3]:
# os.chdir('/content/gdrive/My Drive/InformationRetrieval/Assignment_1')

## Retrieve all the folders

In [4]:
locations = [pos[0] for pos in os.walk(str(os.getcwd())+'/'+'stories'+'/')]
len_locations = len(locations[0])
locations[0] = locations[0][:len_locations-1]

print("Folder Locations = ",locations)

Folder Locations =  ['d:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories', 'd:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories/FARNON', 'd:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories/SRE']


## Load all files from index files

In [5]:
dataset = []

flag = False

for i in locations:
    file = open(i + "/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if flag == False:
        file_name = file_name[2:]
        flag = True

    for j in range(len(file_name)):
        dataset.append((str(i) + "/" + str(file_name[j]), file_title[j]))

N = len (dataset)
print("Total number of files =",N)

Total number of files = 467


## Preprocessing Functions

In [6]:
def l_case(data):
    return np.char.lower(data)
    
def stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_word = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_word = new_word + " " + w
    return np.char.strip(new_word)

def punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    return data

def apostrophe(data):
    return np.char.replace(data, "'", "")

# def convert_numbers(data):
#     tokens = word_tokenize(str(data))
#     new_text = ""
#     for w in tokens:
#         try:
#             w = num2words(int(w))
#         except:
#             a = 0
#         new_text = new_text + " " + w
#     new_text = np.char.replace(new_text, "-", " ")
#     return new_text

# def lemming(data):
#     wordnet_lemmatizer = WordNetLemmatizer()
#     tokens = word_tokenize(str(data))
#     new_word = ""
#     for w in tokens:
#         new_word = new_word + " " + wordnet_lemmatizer.lemmatize(w)
#     return np.char.strip(new_word)


In [7]:
def preprocess(text):
    text = l_case(text)
    text = punctuation(text)
    text = apostrophe(text)
    text = stop_words(text)
    # text = convert_numbers(text)
    return text


## Make Postings

In [8]:
doc = 0
postings = pd.DataFrame()
for i in dataset:
    file = open(i[0], 'r', encoding='utf-8', errors='ignore')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text)
    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            posting = postings[token][0]
            posting.add(doc)
            postings[token][0] = posting
        else:
            postings.insert(value=[{doc}], loc=0, column=token)
    doc += 1

In [9]:
# Check Postings
postings

,retook,superseded,powernodes,ardonite,vortexer,vortexers,optimal,defacing,periis,perii,...,53,west,100,continue,support,need,freeware,project,trial,shareware
0,{466},{466},{466},{466},{466},{466},{466},{466},{466},{466},...,"{0, 196, 356, 169, 170, 44, 76, 206, 435, 310,...","{0, 128, 129, 134, 390, 8, 13, 14, 142, 270, 4...","{0, 136, 264, 394, 41, 42, 170, 47, 435, 438, ...","{0, 6, 11, 16, 26, 39, 40, 47, 48, 50, 58, 63,...","{0, 256, 386, 387, 388, 7, 395, 142, 399, 144,...","{0, 3, 8, 11, 13, 14, 16, 18, 21, 22, 24, 26, ...",{0},"{0, 2, 142, 144, 402, 404, 283, 412, 42, 170, ...","{0, 131, 388, 142, 273, 150, 407, 26, 41, 42, ...","{0, 42, 76, 78, 126}"


## Take queries in input format

In [10]:
flag1=0
query = preprocess(input('Enter Input Query')).tolist().split()
print(query)
    

['telephone', 'paved', 'roads']


In [11]:
print(type(query))

<class 'list'>


## Retrieve Documents

In [80]:
def get_jaccard_values(query):
    jaccard_coefficient = []
    for i in dataset:
        file = open(i[0], 'r', encoding='utf-8', errors='ignore')
        text = file.read().strip()
        file.close()
        preprocessed_text = preprocess(text)
        tokens = word_tokenize(str(preprocessed_text))
        # print(type(tokens))
        intersection_size = len(set(query).intersection(set(tokens)))
        union_size = len(set(query).union(set(tokens)))
        jaccard_coefficient_val = intersection_size/float(union_size)
        print(jaccard_coefficient_val)
        jaccard_coefficient.append(jaccard_coefficient_val)
    print(len(jaccard_coefficient))
    return jaccard_coefficient

## Get Jaccard Coefficient for all docs with Query

In [81]:
jaccard_coefficient = get_jaccard_values(query)

0.0026905829596412557
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0005847953216374269
0.0
0.0
0.0
0.0
0.0
0.0005737234652897303
0.002544529262086514
0.0
0.0
0.0
0.0
0.0
0.0
0.006134969325153374
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0005711022272986865
0.0
0.0002983293556085919
0.0
0.0
0.0012787723785166241
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.003703703703703704
0.0
0.0004941932295527551
0.0
0.00030385900941962927
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0013003901170351106
0.0
0.0
0.0009578544061302681
0.0
0.0006999533364442371
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0010718113612004287
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.000676132521974307
0.0
0.0
0.0008156606851549756
0.0
0.0
0.0003508771929824561
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0007027406886858749
0.0005773672055427252
0.0
0.00281690140845070

In [82]:
jaccard_sorted = sorted(jaccard_coefficient)
# print(jaccardListTemp)
docName = []
print(jaccard_sorted)
top5 = jaccard_sorted[-5:]
print(top5)
for i in range(5):        
    print(top5[4-i]," and index is ",jaccard_coefficient.index(top5[4-i]))
    index = jaccard_coefficient.index(top5[4-i])
    head, tail = os.path.split(dataset[index][0])
    tail.strip()
    docName.append(tail.strip())
    docName.append(" with Jaccard Similarity Value:")
    docName.append(top5[4-i])
    docName.append("\n")
    jaccard_coefficient[index] = 0
print("\nList of Top Documents wrt Jaccard Coefficient for i/p query is:")
print(*docName)
print("\n-----------------------------------------------\n-----------------------------------------------\n")

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,